In [2]:
# Dependencies and Setup
import pandas as pd
import requests
import gmaps
import matplotlib


# Import API key
from config import g_key

# Configure gmaps API key
gmaps.configure(api_key=g_key)

In [3]:
# 1. Import the WeatherPy_database.csv file. 
city_data_df = pd.read_csv("C:/Users/remem/Desktop/Boot-Camp/module-6/World_Weather_Analysis/Weather_Data/WeatherPy_data.csv")
city_data_df.head()
    

,City_ID,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date,Current Description
0,0,Rikitea,-23.1203,-134.9692,76.17,71,50,9.33,PF,2021-12-04 23:15:12,scattered clouds
1,1,Mataura,-46.1927,168.8643,55.47,91,100,6.17,NZ,2021-12-04 23:13:55,overcast clouds
2,2,Vaini,-21.2000,-175.2000,84.36,74,75,14.97,TO,2021-12-04 23:13:58,light rain
3,3,Hermanus,-34.4187,19.2345,63.90,93,100,17.45,ZA,2021-12-04 23:14:02,heavy intensity rain
4,4,Paamiut,61.9940,-49.6678,35.91,76,99,44.38,GL,2021-12-04 23:19:22,light rain


In [4]:
# 2. Prompt the user to enter minimum and maximum temperature criteria 

min_temp = float(input('What is the minimum temperature you wuold like for your trip? '))
max_temp = float(input('What is the maximum temperature you would like for your trip? '))

What is the minimum temperature you wuold like for your trip? 20
What is the maximum temperature you would like for your trip? 50


In [5]:
# 3. Filter the city_data_df DataFrame using the input statements to create a new DataFrame using the loc method.

preferred_cities_df = city_data_df.loc[(city_data_df['Max Temp'] <= max_temp) &
                                      (city_data_df['Max Temp'] >= min_temp)]

preferred_cities_df.head(10)

,City_ID,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date,Current Description
4,4,Paamiut,61.9940,-49.6678,35.91,76,99,44.38,GL,2021-12-04 23:19:22,light rain
9,9,Kizner,56.2749,51.5080,27.34,89,89,9.26,RU,2021-12-04 23:19:23,overcast clouds
12,12,Ilulissat,69.2167,-51.1000,35.62,55,100,39.12,GL,2021-12-04 23:19:24,overcast clouds
27,27,Bethel,41.3712,-73.4140,43.65,72,1,0.00,US,2021-12-04 23:19:29,clear sky
28,28,Lasa,46.6166,10.7002,33.26,95,100,1.81,IT,2021-12-04 23:19:30,snow
33,33,Ushuaia,-54.8000,-68.3000,49.66,53,40,18.41,AR,2021-12-04 23:13:51,scattered clouds
34,34,Albany,42.6001,-73.9662,40.68,80,87,7.27,US,2021-12-04 23:10:56,overcast clouds
37,37,Kodiak,57.7900,-152.4072,26.11,68,40,9.22,US,2021-12-04 23:17:22,scattered clouds
39,39,Taicheng,38.7310,113.2280,29.61,27,0,2.33,CN,2021-12-04 23:19:33,clear sky
44,44,Onalaska,43.8844,-91.2351,34.32,56,90,5.75,US,2021-12-04 23:19:34,overcast clouds


In [6]:
# 4a. Determine if there are any empty rows.

preferred_cities_df.isnull().sum()

City_ID                0
City                   0
Lat                    0
Lng                    0
Max Temp               0
Humidity               0
Cloudiness             0
Wind Speed             0
Country                0
Date                   0
Current Description    0
dtype: int64

In [7]:
# 4b. Drop any empty rows and create a new DataFrame that doesn’t have empty rows.

preferred_cities_clean_df = preferred_cities_df.dropna()
preferred_cities_clean_df.isnull().sum()

City_ID                0
City                   0
Lat                    0
Lng                    0
Max Temp               0
Humidity               0
Cloudiness             0
Wind Speed             0
Country                0
Date                   0
Current Description    0
dtype: int64

In [8]:
# 5a. Create DataFrame called hotel_df to store hotel names along with city, country, max temp, and coordinates.
hotel_df = preferred_cities_clean_df[["City", "Country", "Max Temp", "Current Description", "Lat", "Lng"]].copy()

# 5b. Create a new column "Hotel Name"
hotel_df["Hotel Name"] = ""
hotel_df.head(10)

,City,Country,Max Temp,Current Description,Lat,Lng,Hotel Name
4,Paamiut,GL,35.91,light rain,61.9940,-49.6678,
9,Kizner,RU,27.34,overcast clouds,56.2749,51.5080,
12,Ilulissat,GL,35.62,overcast clouds,69.2167,-51.1000,
27,Bethel,US,43.65,clear sky,41.3712,-73.4140,
28,Lasa,IT,33.26,snow,46.6166,10.7002,
33,Ushuaia,AR,49.66,scattered clouds,-54.8000,-68.3000,
34,Albany,US,40.68,overcast clouds,42.6001,-73.9662,
37,Kodiak,US,26.11,scattered clouds,57.7900,-152.4072,
39,Taicheng,CN,29.61,clear sky,38.7310,113.2280,
44,Onalaska,US,34.32,overcast clouds,43.8844,-91.2351,


In [9]:
# 6a. Set parameters to search for hotels with 5000 meters.
params = {
    "radius": 5000,
    "type": "lodging",
    "key": g_key
}

# 6b. Iterate through the hotel DataFrame.
for index, row in hotel_df.iterrows():

    # 6c. Get latitude and longitude from DataFrame
    lat = row['Lat']
    lng = row['Lng']
    params['location'] = f"{lat},{lng}"

    
    # 6d. Set up the base URL for the Google Directions API to get JSON data.
    base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"
    

    # 6e. Make request and retrieve the JSON data from the search.
    hotels = requests.get(base_url, params=params).json()
   
    
    # 6f. Get the first hotel from the results and store the name, if a hotel isn't found skip the city.
    try:
        hotel_df.loc[index, 'Hotel Name'] = hotels['results'][0]['name']
    except (IndexError):
        print('hotel not found... skipping.')

hotel not found... skipping.
hotel not found... skipping.
hotel not found... skipping.
hotel not found... skipping.
hotel not found... skipping.
hotel not found... skipping.
hotel not found... skipping.
hotel not found... skipping.
hotel not found... skipping.
hotel not found... skipping.
hotel not found... skipping.
hotel not found... skipping.
hotel not found... skipping.
hotel not found... skipping.
hotel not found... skipping.


In [10]:
# check it
hotel_df.head(25)

,City,Country,Max Temp,Current Description,Lat,Lng,Hotel Name
4,Paamiut,GL,35.91,light rain,61.9940,-49.6678,Pistut
9,Kizner,RU,27.34,overcast clouds,56.2749,51.5080,Gostinitsa Intera
12,Ilulissat,GL,35.62,overcast clouds,69.2167,-51.1000,Hotel Arctic
27,Bethel,US,43.65,clear sky,41.3712,-73.4140,Courtyard by Marriott Danbury
28,Lasa,IT,33.26,snow,46.6166,10.7002,Pension Feldgärtenhof
33,Ushuaia,AR,49.66,scattered clouds,-54.8000,-68.3000,Albatros Hotel
34,Albany,US,40.68,overcast clouds,42.6001,-73.9662,
37,Kodiak,US,26.11,scattered clouds,57.7900,-152.4072,Best Western Kodiak Inn And Convention Center
39,Taicheng,CN,29.61,clear sky,38.7310,113.2280,五台山康旅酒店
44,Onalaska,US,34.32,overcast clouds,43.8844,-91.2351,Stoney Creek Hotel La Crosse - Onalaska


In [11]:
# 7. Drop the rows where there is no Hotel Name.

# define NaN
nan_value = float("NaN")
# in new dataframe, replace empty hotel column cells with NaN
clean_hotel_df = hotel_df.replace("", nan_value)
# drop rows with NaN values from new dataframe
clean_hotel_df = clean_hotel_df.dropna()
# check it
clean_hotel_df.head(25)

# link below is citation for the coding method above in cell
# https://www.kite.com/python/answers/how-to-drop-empty-rows-from-a-pandas-dataframe-in-python

,City,Country,Max Temp,Current Description,Lat,Lng,Hotel Name
4,Paamiut,GL,35.91,light rain,61.9940,-49.6678,Pistut
9,Kizner,RU,27.34,overcast clouds,56.2749,51.5080,Gostinitsa Intera
12,Ilulissat,GL,35.62,overcast clouds,69.2167,-51.1000,Hotel Arctic
27,Bethel,US,43.65,clear sky,41.3712,-73.4140,Courtyard by Marriott Danbury
28,Lasa,IT,33.26,snow,46.6166,10.7002,Pension Feldgärtenhof
33,Ushuaia,AR,49.66,scattered clouds,-54.8000,-68.3000,Albatros Hotel
37,Kodiak,US,26.11,scattered clouds,57.7900,-152.4072,Best Western Kodiak Inn And Convention Center
39,Taicheng,CN,29.61,clear sky,38.7310,113.2280,五台山康旅酒店
44,Onalaska,US,34.32,overcast clouds,43.8844,-91.2351,Stoney Creek Hotel La Crosse - Onalaska
45,Beian,NO,30.29,broken clouds,63.6559,9.5683,"Fjord-Fishing AS, Property & Housing Co."


In [12]:
# 8a. Create the output File (CSV)
output_data_file = 'WeatherPy_vacation.csv'

# 8b. Export the City_Data into a csv
clean_hotel_df.to_csv(output_data_file, index_label="City_ID")

In [13]:
# 9. Using the template add city name, the country code, the weather description and maximum temperature for the city.
info_box_template = """
<dl>
<dt>Hotel Name</dt><dd>{Hotel Name}</dd>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
<dt>Current Description</dt><dd>{Current Description}</dd>
<dt>Max Temp</dt><dd>{Max Temp} °F</dd>
</dl>
"""

# 10a. Get the data from each row and add it to the formatting template and store the data in a list.
hotel_info = [info_box_template.format(**row) for index, row in clean_hotel_df.iterrows()]

# 10b. Get the latitude and longitude from each row and store in a new DataFrame.
locations = clean_hotel_df[["Lat", "Lng"]]

In [14]:
# 11a. Add a marker layer for each city to the map. 
fig = gmaps.figure(center=(30.0, 31.0), zoom_level=1.5)
heat_layer = gmaps.heatmap_layer(locations, dissipating=False,
                                max_intensity=300, point_radius=4)
marker_layer = gmaps.marker_layer(locations, info_box_content=hotel_info)

# 11b. Display the figure
fig.add_layer(heat_layer)
fig.add_layer(marker_layer)
fig

Figure(layout=FigureLayout(height='420px'))